Здесь будет производится скачивание и парсинг данных с сайта auto.ru

Задача: Оценка рыночной стоимости мотоцикла с пробегом 

Данные: https://auto.ru/moskva/mototsikly/all/ 
Целевая переменная: цена в рублях (!)

Признаки: 
1. Фирма изготовитель (!)
2. Пробег (!)
3. Тип 
4. Год выпуска 

5. Объем двигателя 
6. Мощность двигателя 
7. Тип двигателя 
8. Число цилиндров 
9. Число тактов 
10. Число передач 
11. Тип привода 
12. Наличие ABS 
13. Наличие электростартера 
14. Цвет 

15. Состояние 
16. Таможня 
17. Частное лицо или дилер 
18. Возможен ли торг 
19. Расстояние от Москвы 

Можно попробовать: 
20. Дата публикации объявления 
21. Число просмотров объявления 
22. Число фотографий в объявлении 
23. Длина комментария продавца

Как они будут называться в таблице, разделитель ',': 

0. price

1. manufacturer
2. mileage 
3. motorcycle_type
4. year 

5. engine_capacity
6. engine_power
7. engine_type 
8. cylinders 
9. strokes 
10. gears 
11. type_of_drive 
12. abs
13. starter 
14. color 

15. damaged (1 если требует ремонта, 0 если нет)
16. сustoms (1 - растаможен)
17. private (1 - частное лицо) 
18. bargaining - (1 если возможен торг) 
19. distance_from_moscow 
 
20. date 
21. visits 
22. photos 
23. length_of_comment

Алгоритм:
1. Скачать все страницы (есть номер страницы, если забанят - знаю, откуда продолжить)
2. Получить из них список ссылкок (просто парсинг)
3. Потом идти по каждой ссылке (есть номер ссылки, если забанят - знаю, откуда продолжить)

<b>Импорт библиотек</b>

In [1]:
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

<b>Класс, описывающий мотоцикл</b>

In [2]:
class Motorcycle:
    def __init__(self, soup_motorcycle, index=None):
        self.soup = soup_motorcycle
        if (index is None):
            self.index = 0
        else:
            self.index = index
            
        self.warning_string_ = "Index: " + str(self.index) + "; Warning: "
        
    def warning_(self, string):
        print(self.warning_string_ + string)
        
    def parse_information(self):
        self.price = self.get_price()
        if (self.price is None):
            return False
        
        self.manufacturer = self.get_manufacturer()
        if (self.manufacturer is None):
            return False
        
        self.mileage = self.get_mileage()
        if (self.mileage is None):
            return False
        
        self.year = self.get_year()
        
        # аналогично дописать для всех фичей
        # проверка на None в этой функции для других фичей не нужна 
        # (я выделил самые главные)
        # 
        return True
    
        
    def get_price(self):
        price = self.soup.select_one('h4.card__price-rur')
        print(price)
        if (price is not None):
            price = price.text
        else:
            self.warning_('price not found')
            price = None
        return price
    
    
    def get_manufacturer(self):
        manufacturer = self.soup.select('.link.link_theme_auto.link__control.i-bem')
        if (manufacturer is not None):
            manufacturer = manufacturer[5].text
        else:
            self.warning_('manufacturer not found')
            manufacturer = None
        return manufacturer
    
    def get_mileage(self):
        mileage = self.soup.select('dl.card__info > dd.card__info-value.card__info-value_bold')
        if (mileage is not None):
            mileage = manufacturer[1].text
        else:
            self.warning_('mileage not found')
            mileage = None
        return mileage
    
    def get_year(self):
        year = self.soup.select('dl.card__info > dd.card__info-value.card__info-value_bold')
        if (year is not None):
            year = manufacturer[0].text
        else:
            self.warning_('year not found')
            year = None
        return year
    
    def __repr__(self):
         return "Motorcycle class"
        
    def __str__(self):
        string = str(self.price) + ',' + str(self.manufacturer) + ',' + \
                    str(self.mileage) + ',' + str(self.year)
        return string

<b>Информация по подключению</b>

In [3]:
url_page_template = '''https://auto.ru/moskva/mototsikly/all/?beaten=1&custom_state_key=CLEARED
                    &geo_id=213geo_radius=1000&image=true&sort_offers=cr_date-DESC&top_days=off
                    &currency=RUR&output_type=list&page_num_offers='''

In [16]:
# Выбрать нужное
# Сайт с прокси: http://spys.one/proxys/RU/

proxies = None
proxies = { 'https': '5.53.19.82:56907' }

In [5]:
#user_agent = UserAgent()
#user_agent.update()
#headers = {'User-Agent': user_agent.chrome}

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'}

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\Users\Overdrive\Anaconda3\lib\urllib\request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "C:\Users\Overdrive\Anaconda3\lib\http\client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\Users\Overdrive\Anaconda3\lib\http\client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "C:\Users\Overdrive\Anaconda3\lib\http\client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "C:\Users\Overdrive\Anaconda3\lib\http\client.py", line 1026, in _send_output
    self.send(msg)
  File "C:\Users\Overdrive\Anaconda3\lib\http\client.py", line 964, in send
    self.connect()
  File "C:\Users\Overdrive\Anaconda3\lib\http\client.py", line 936, in 

<b>Вспомогательные функции для подключения</b>

In [6]:
def is_banned(soup):
    title = soup.select_one('h1.title')
    if (title is not None):
        if (title.text == 'ой…'):
            #print(soup)
            return True
    else:
        return False

In [7]:
def is_page_found(soup):
    title = soup.select_one('div.page-title')
    if (title is not None):
        if (title.text == 'Страница не найдена'):
            return False
    else:
        return True

In [8]:
def get_links(url_page, proxies=None, headers=None, sleep=False, sleep_seconds=10):
    html_page = requests.get(url_page, proxies=proxies, headers=headers).content
    soup = BeautifulSoup(html_page, 'html.parser')
    
    if (is_banned(soup)):
        print("BANNED")
        return None
    
    if (not is_page_found(soup)):
        print("Page not found")
        return None
    
    if (sleep):
        time.sleep(np.random.randint(sleep_seconds))
          
    return soup.select('.link.link_theme_auto.listing-item__link.link__control.i-bem')

In [9]:
def get_motorcycle_soup(url_motorcycle, proxies=None, headers=None, sleep=False, sleep_seconds=10):
    html_motorcycle = requests.get(url_motorcycle, proxies=proxies, headers=headers).content
    soup_motorcycle = BeautifulSoup(html_motorcycle, 'html.parser')
        
    if (is_banned(soup_motorcycle)):
        #print(soup_motorcycle)
        print("BANNED")
        return None
    
    if (not is_page_found(soup_motorcycle)):
        #print(soup_motorcycle)
        print("Page not found")
        return None
    
    if (sleep):
        time.sleep(np.random.randint(sleep_seconds))
        
    return soup_motorcycle

<b>Создание списка ссылок</b>

In [10]:
if (False): # чтобы не запускалось при нажатии Run all
    last_page = 99

    with open('data/current_page.txt', 'r') as file:
        current_page = int(file.readline())
        current_link = int(file.readline())

    with open('data/links.txt', 'a') as f_output:
        for page in range(current_page, last_page+1):
            links = get_links(url_page_template + str(page), proxies, headers, True)

            if (links is None):
                print("Error while getting motorcycle links")
                current_page = page
                print("Page: " + str(current_page))
                print("Link: " + str(current_link))
                break

            for link in links:
                print(str(current_link) + '. '+ link['href'], file=f_output)
                current_link += 1

            current_page = page + 1

    with open('data/current_page.txt', 'w') as f_output:
        print(current_page, file=f_output)
        print(current_link, file=f_output)

<b>Основной цикл (не работает пока)</b>

In [11]:
if (False): # чтобы не запускалось при нажатии Run all

    with open('data/current_parce_link.txt', 'r') as file:
        current_parce_link = int(file.readline())

    with open('data/links.txt', 'r') as links_file:
        with open('data/motorcycles.txt', 'a') as motorcycle_file:
            for i, link in enumerate(links_file):
                if (i < current_parce_link):
                    continue

                link = link[link.find('h'):]

                soup = get_motorcycle_soup(link, proxies, headers, True)
                if (soup is None):
                    print("Error while getting motorcycle information")
                    print("Link: " + str(current_parce_link))
                    break

                motorcycle = Motorcycle(soup, current_parce_link)
                if (motorcycle.parse_information()):
                    print(motorcycle, file=motorcycle_file)
                else:
                    print("Error while parsing motorcycle information")
                    print("Link: " + str(current_parce_link))

                current_parce_link += 1

    with open('data/current_parce_link.txt', 'w') as f_output:
        print(current_parce_link, file=f_output)

Все, что ниже - черновик, для того, чтобы что то отдельное потестить

Если банит, то можно скачать один раз в отдельной ячейке, и потом просто по ней делать (чтобы не перекачивать)

In [14]:
number_of_link = 1
with open('data/links.txt', 'r') as links_file:
    for i in range(number_of_link):
        link = links_file.readline()
        
    link = link[link.find('h'):]
    #link = link[:8] + 'www.' + link[8:]
    print("Link: " + link)
    
    soup = get_motorcycle_soup(str(link), proxies, headers, True)

Link: https://auto.ru/mototsikly/used/sale/ducati/diavel_carbon/2814439-9a82eb97/

BANNED


Ячейка выше - один раз запустили, скачали

Теперь тестируем парсинг снизу

In [13]:
motorcycle = Motorcycle(soup)
print(soup)

if (motorcycle.parse_information()):
    print(motorcycle)

None


AttributeError: 'NoneType' object has no attribute 'select_one'

In [17]:
html_motorcycle = requests.get("https://auto.ru/mototsikly/used/sale/ducati/diavel_carbon/2814439-9a82eb97/", proxies).content
soup_motorcycle = BeautifulSoup(html_motorcycle, 'html.parser')

if (is_banned(soup_motorcycle)):
    #print(soup_motorcycle)
    print("BANNED")
    
if (not is_page_found(soup_motorcycle)):
    #print(soup_motorcycle)
    print("Page not found")

BANNED
